In [1]:
"""
Use of same experimental setup as for our Probabilistic Suffix Prediction.

Reimplementation for comparison: 
- Paper: Weytjens, Hans, and Jochen De Weerdt. "Learning uncertainty with artificial neural networks for predictive process monitoring." Applied Soft Computing 125 (2022): 109134.
- Github (code) from: https://github.com/hansweytjens/uncertainty-remaining_time/blob/main/LSTM.ipynb
"""

'\nUse of same experimental setup as for our Probabilistic Suffix Prediction.\n\nReimplementation for comparison: \n- Paper: Weytjens, Hans, and Jochen De Weerdt. "Learning uncertainty with artificial neural networks for predictive process monitoring." Applied Soft Computing 125 (2022): 109134.\n- Github (code) from: https://github.com/hansweytjens/uncertainty-remaining_time/blob/main/LSTM.ipynb\n'

# Imports

In [2]:
import importlib
import sys
import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')

# Data

### Load Data Files

In [3]:
# Path to your pickle file (saved with torch.save)
file_path_train = '../../../../../../encoded_data/compare_weytjens/BPIC_2017_all_5_train.pkl'
# Load the dataset using torch.load
bpic17_train_dataset = torch.load(file_path_train, weights_only=False)
# Check the type of the loaded dataset
print(type(bpic17_train_dataset))

# Path to your pickle file (saved with torch.save)
file_path_val = '../../../../../../encoded_data/compare_weytjens/BPIC_2017_all_5_val.pkl'
# Load the dataset using torch.load
bpic17_val_dataset = torch.load(file_path_val, weights_only=False)
# Check the type of the loaded dataset
print(type(bpic17_val_dataset))


<class 'event_log_loader.new_event_log_loader.EventLogDataset'>


<class 'event_log_loader.new_event_log_loader.EventLogDataset'>


### Train Data Insights

In [4]:
# BPIC 17 Dataset Categories, Features:
bpic17_all_categories = bpic17_train_dataset.all_categories

bpic17_all_categories_cat = bpic17_all_categories[0]
print(bpic17_all_categories_cat)

bpic17_all_categories_num = bpic17_all_categories[1]
print(bpic17_all_categories_num)

for i, cat in enumerate(bpic17_all_categories_cat):
     print(f"bpic17 (5) Categorical feature: {cat[0]}, Index position in categorical data list: {i}")
     print(f"bpic17 (5) Total Amount of Category labels: {cat[1]}")

print('\n')    

for i, num in enumerate(bpic17_all_categories_num):
     print(f"bpic17 (5) Numerical feature: {num[0]}, Index position in categorical data list: {i}")
     print(f"bpic17 (5) Amount Numerical: {num[1]}")
     
# Get concept_name id:
# 
concept_name = 'concept:name'
concept_name_id = [i for i, cat in enumerate(bpic17_all_categories[0]) if cat[0] == concept_name][0]

print("ID concet name in cat list: ", concept_name_id)
    
# Id of EOS token in activity
eos_value = 'EOS'
eos_id = [v for k, v in bpic17_all_categories[0][concept_name_id][2].items() if k == eos_value][0]
# Get EOS id of concept name list:

print("ID EOS in concept name tensor: ", eos_id)


[('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27})]
[('case_elapsed_time', 1, {})]
bpic17 (5) Categorical feature: concept:name, Index position in categorical data list: 0
bpic17 (5) Total Amount of Category labels: 28


bpic17 (5) Numerical feature: case_elapsed_time, Index position in categorical data list: 0
bpic17 (5) Amount Numerical: 1
ID concet name in cat list:  0
ID EOS in concept name tensor:  11


### Input Features for Encoder and Decoder

In [5]:
# Create lists with name of Model features (input)
model_feat_cat = ['concept:name']
model_feat_num = ['case_elapsed_time']
model_feat = [model_feat_cat, model_feat_num]

print("Features model: ", model_feat)


Features model:  [['concept:name'], ['case_elapsed_time']]


# Model

In [6]:
import stochasticLSTM.model
importlib.reload(stochasticLSTM.model)
from stochasticLSTM.model import StochasticLSTMWeytjens

"""
Specific model parameters from paper: 
"""

# Size hidden layer
hidden_size=10

# Number of LSTM cells
num_layers=2

# One numerical output to predict
input_size=1

# Fixed Dropout probability 
p_fix=0.1

# Lambda for L2 (weight, bias, dropout) regularization: According to formula: 1/2N
regularization_term = 0.1

# Hans Weytjens LSTM model
model = StochasticLSTMWeytjens(data_set_categories=bpic17_all_categories,
                               model_feat=model_feat,
                               hidden_size=hidden_size,
                               num_layers=num_layers,
                               input_size=input_size,
                               weight_reg=regularization_term,
                               p_fix=p_fix,
                               device='cuda')

Data set categories:  ([('concept:name', 28, {'A_Accepted': 1, 'A_Cancelled': 2, 'A_Complete': 3, 'A_Concept': 4, 'A_Create Application': 5, 'A_Denied': 6, 'A_Incomplete': 7, 'A_Pending': 8, 'A_Submitted': 9, 'A_Validating': 10, 'EOS': 11, 'O_Accepted': 12, 'O_Cancelled': 13, 'O_Create Offer': 14, 'O_Created': 15, 'O_Refused': 16, 'O_Returned': 17, 'O_Sent (mail and online)': 18, 'O_Sent (online only)': 19, 'W_Assess potential fraud': 20, 'W_Call after offers': 21, 'W_Call incomplete files': 22, 'W_Complete application': 23, 'W_Handle leads': 24, 'W_Personal Loan collection': 25, 'W_Shortened completion ': 26, 'W_Validate application': 27})], [('case_elapsed_time', 1, {})])
Model input features:  [['concept:name'], ['case_elapsed_time']]


Embeddings:  ModuleList(
  (0): Embedding(28, 10)
)
Total embedding feature size:  10
Input feature size:  11
Cells hidden size:  10
Number of LSTM layer:  2
Dropout rate:  0.1


Output feature list of dicts (featue name, tensor index in dataset):  {

# Loss Object Creation

In [7]:
import loss_hans.losses
importlib.reload(loss_hans.losses)
from loss_hans.losses import Loss

loss_obj = Loss()

# Training Configuration

In [8]:
import training.train
importlib.reload(training.train)
from training.train import Training

from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(comment="Full_BPIC17_weytjens_rem_time")

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

"""
Parameter of Probabilistic Suffix Prediction experimental design, to ensure fair comparison:
"""

# Start learning rate
learning_rate = 1e-6

# Optimizer and Scheduler
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay=0)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, min_lr=1e-10)

# Epochs
num_epochs = 100

# Batch size
batch_size = 256

# shuffle data
shuffle = True

optimize_values = {"regularization_term":regularization_term,
                   "optimizer":optimizer,
                   "scheduler": scheduler,
                   "epochs":num_epochs,
                   "mini_batches":batch_size,
                   "shuffle": shuffle}

number_tasks = len(model_feat)

trainer = Training(model=model,
                   device=device,
                   data_train=bpic17_train_dataset,
                   data_val=bpic17_val_dataset,
                   concept_name_id=concept_name_id,
                   eos_id=eos_id,
                   loss_obj=loss_obj,
                   optimize_values=optimize_values,
                   writer=writer,
                   save_model_n_th_epoch=1,
                   saving_path="BPIC17_weytjens_rem_time_1_suffix_length5.pkl")

# Train the model:
trainer.train()

Device:  cuda
Optimizer:  Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-06
    maximize: False
    weight_decay: 0
)
Scheduler:  <torch.optim.lr_scheduler.ReduceLROnPlateau object at 0x7fb6f1764590>
Epochs:  100
Mini baches:  256
Shuffle batched dataset:  True


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch [1/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 14.1709


Validation: Avg Standard Validation Loss: 2.5129
Validation: Avg Attenuated Validation Loss: 2.3951
Validation Loss for Scheduler: 2.5129
saving model


Epoch [2/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 14.1392


...

Validation: Avg Standard Validation Loss: 0.9064
Validation: Avg Attenuated Validation Loss: 0.9299
Validation Loss for Scheduler: 0.9064
saving model


Epoch [99/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 14.5305


Validation: Avg Standard Validation Loss: 0.9035
Validation: Avg Attenuated Validation Loss: 0.9277
Validation Loss for Scheduler: 0.9035
saving model


Epoch [100/100], Learning Rate: 1e-06
Training: Avg Attenuated Training Loss: 14.5536


Validation: Avg Standard Validation Loss: 0.8885
Validation: Avg Attenuated Validation Loss: 0.9144
Validation Loss for Scheduler: 0.8885
saving model
Training complete.
Model saved to path: BPIC17_weytjens_rem_time_1_suffix_length5.pkl
